# Batch computation

This notebook computes event rate at T2K and PS191 throughout the parameter space. Sometimes we compute the lifetime as well.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from scipy import interpolate

import pathos.multiprocessing as mp
from pathos.multiprocessing import ProcessingPool as Pool

from functools import partial

from particle import *
from particle import literals as lp

from hnl_apps import *
from hnl_apps.plot_tools import *
from hnl_apps.const import *

from Nlimits import *
from Nlimits.constraint_dict import *
from Nlimits.plotter import *

In [3]:
# set up grid
gridx = 100
gridy = 50

M4 = np.linspace(0.02,(m_neutral_kaon - m_mu), gridx)
USQR = np.logspace(-9,-2, gridy) 
X,Y = np.meshgrid(M4,USQR)
m4_grid, usqr_grid = X.flatten(), Y.flatten()

### T2K

In [21]:
# compute rate and lifetimes
tot_cores = mp.cpu_count()
pool = Pool(tot_cores)
############

get_event_partial = partial(hnl_tools.get_event_rate, flavor_struct=[0.0,1.0,0.0], detector = nd280)
event_rate_weak = pool.map(get_event_partial, ((m4, usqr) for m4, usqr in zip(m4_grid, usqr_grid)))

get_lifetime_partial = partial(hnl_tools.get_lifetime, flavor_struct=[0.0,1.0,0.0])
lifetime_weak = pool.map(get_lifetime_partial, ((m4, usqr) for m4, usqr in zip(m4_grid, usqr_grid)))

############
pool.terminate()
pool.restart()

<multiprocess.pool.Pool state=RUN pool_size=8>

In [22]:
# Save to file
table_T2K_minimal = np.array([m4_grid, usqr_grid, lifetime_weak, event_rate_weak])
np.save("limits_computed/T2K_minimal_Umu4",table_T2K_minimal)
np.savetxt("limits_computed/T2K_minimal_Umu4.dat",table_T2K_minimal, header=" m4(GeV) USQR tau0(s) event_rate")

### PS191

In [96]:
# compute rate and lifetimes
tot_cores = mp.cpu_count()
pool = Pool(tot_cores)
############

get_event_partial = partial(hnl_tools.get_event_rate, flavor_struct=[0.0,1.0,0.0], detector = ps191)
event_rate_weak_ps191 = pool.map(get_event_partial, ((m4, usqr) for m4, usqr in zip(m4_grid, usqr_grid)))

############
pool.terminate()
pool.restart()


<multiprocess.pool.Pool state=RUN pool_size=8>

In [97]:
# Save to file
table_PS191_minimal = np.array([m4_grid, usqr_grid, lifetime_weak, event_rate_weak_ps191])
np.save("limits_computed/PS191_minimal_Umu4",table_PS191_minimal)
np.savetxt("limits_computed/PS191_minimal_Umu4.dat",table_PS191_minimal, header=" m4(GeV) USQR tau0(s) event_rate")

# Magnetic moment transition

## d = 1 PeV^-1 

In [98]:
# set up grid
gridx = 100
gridy = 50

M4 = np.linspace(0.02,(m_neutral_kaon - m_mu), gridx)
USQR = np.logspace(-14,-4, gridy) 
X,Y = np.meshgrid(M4,USQR)
m4_grid, usqr_grid = X.flatten(), Y.flatten()

dipoles = {'dip_mu4': 5e-7}

### T2K

In [99]:
# compute rate and lifetimes
tot_cores = mp.cpu_count()
pool = Pool(tot_cores)
############

get_event_partial = partial(hnl_tools.get_event_rate, flavor_struct=[0.0,1.0,0.0], dipoles=dipoles, detector = nd280)
event_rate_dipole = pool.map(get_event_partial, ((m4, usqr) for m4, usqr in zip(m4_grid, usqr_grid)))
get_lifetime_partial = partial(hnl_tools.get_lifetime, flavor_struct=[0.0,1.0,0.0], dipoles=dipoles)
lifetime_dipole = pool.map(get_lifetime_partial, ((m4, usqr) for m4, usqr in zip(m4_grid, usqr_grid)))

############
pool.terminate()
pool.restart()

<multiprocess.pool.Pool state=RUN pool_size=8>

In [100]:
# Save to file
table_T2K_magmoment = np.array([m4_grid, usqr_grid, lifetime_dipole, event_rate_dipole])
np.save("limits_computed/T2K_mag_mom_Umu4",table_T2K_magmoment)
np.savetxt("limits_computed/T2K_mag_mom_Umu4.dat",table_T2K_magmoment, header=" m4(GeV) USQR tau0(s) event_rate")

### PS191

In [101]:
# compute rate and lifetimes
tot_cores = mp.cpu_count()
pool = Pool(tot_cores)
############

get_event_partial = partial(hnl_tools.get_event_rate, flavor_struct=[0.0,1.0,0.0], dipoles=dipoles, detector = ps191)
event_rate_dipole_PS191 = pool.map(get_event_partial, ((m4, usqr) for m4, usqr in zip(m4_grid, usqr_grid)))
get_lifetime_partial = partial(hnl_tools.get_lifetime, flavor_struct=[0.0,1.0,0.0], dipoles=dipoles)
lifetime_dipole_PS191 = pool.map(get_lifetime_partial, ((m4, usqr) for m4, usqr in zip(m4_grid, usqr_grid)))

############
pool.terminate()
pool.restart()

<multiprocess.pool.Pool state=RUN pool_size=8>

In [102]:
# Save to file
table_PS191_mag_mom = np.array([m4_grid, usqr_grid, lifetime_dipole_PS191, event_rate_dipole_PS191])
np.save("limits_computed/PS191_mag_mom_Umu4",table_PS191_mag_mom)
np.savetxt("limits_computed/PS191_mag_mom_Umu4.dat",table_PS191_mag_mom, header=" m4(GeV) USQR tau0(s) event_rate")

In [12]:
print(hnl_tools.get_event_rate((0.05,1e-6), flavor_struct=[0.0,1.0,0.0], dipoles={'dip_mu4': 5e-9}, detector = ps191))
print(hnl_tools.get_event_rate((0.05,1e-6), flavor_struct=[0.0,1.0,0.0], dipoles={'dip_mu4': 5e-9}, detector = nd280))


0.7075940856030256
1.2866704194221013


## Schwetz-mania -- mixing and mass

d = 5e-9 GeV^-1

In [105]:
# set up grid
gridx = 100
gridy = 50

M4 = np.linspace(0.02,(m_neutral_kaon - m_mu), gridx)
USQR = np.logspace(-12,-4, gridy) 
X,Y = np.meshgrid(M4,USQR)
m4_grid, usqr_grid = X.flatten(), Y.flatten()

dipoles = {'dip_mu4': 5e-9}

In [106]:
# compute rate and lifetimes
tot_cores = mp.cpu_count()
pool = Pool(tot_cores)
############

get_event_partial = partial(hnl_tools.get_event_rate, flavor_struct=[0.0,1.0,0.0], dipoles=dipoles, detector=nd280)
event_rate_dipole = pool.map(get_event_partial, ((m4, usqr) for m4, usqr in zip(m4_grid, usqr_grid)))
get_lifetime_partial = partial(hnl_tools.get_lifetime, flavor_struct=[0.0,1.0,0.0], dipoles=dipoles)
lifetime_dipole = pool.map(get_lifetime_partial, ((m4, usqr) for m4, usqr in zip(m4_grid, usqr_grid)))

############
pool.terminate()
pool.restart()

<multiprocess.pool.Pool state=RUN pool_size=8>

In [107]:
# Save to file
table_T2K_magmoment = np.array([m4_grid, usqr_grid, lifetime_dipole, event_rate_dipole])
np.save("limits_computed/T2K_mag_mom_schwetz_Umu4",table_T2K_magmoment)
np.savetxt("limits_computed/T2K_mag_mom_schwetz_Umu4.dat",table_T2K_magmoment, header=" m4(GeV) USQR tau0(s) event_rate")

### PS191

In [108]:
# compute rate and lifetimes
tot_cores = mp.cpu_count()
pool = Pool(tot_cores)
############

get_event_partial = partial(hnl_tools.get_event_rate, flavor_struct=[0.0,1.0,0.0], dipoles=dipoles, detector = ps191)
event_rate_dipole_PS191 = pool.map(get_event_partial, ((m4, usqr) for m4, usqr in zip(m4_grid, usqr_grid)))
get_lifetime_partial = partial(hnl_tools.get_lifetime, flavor_struct=[0.0,1.0,0.0], dipoles=dipoles)
lifetime_dipole_PS191 = pool.map(get_lifetime_partial, ((m4, usqr) for m4, usqr in zip(m4_grid, usqr_grid)))

############
pool.terminate()
pool.restart()

<multiprocess.pool.Pool state=RUN pool_size=8>

In [109]:
# Save to file
table_PS191_mag_mom = np.array([m4_grid, usqr_grid, lifetime_dipole_PS191, event_rate_dipole_PS191])
np.save("limits_computed/PS191_mag_mom_schwetz_Umu4",table_PS191_mag_mom)
np.savetxt("limits_computed/PS191_mag_mom_schwetz_Umu4.dat",table_PS191_mag_mom, header=" m4(GeV) USQR tau0(s) event_rate")

# Schwetz -- dipole  vs  mixing

In [110]:
# set up grid
gridx = 100
gridy = 50

M4 = 0.250

dmu = np.logspace(-10,-6, gridx)
USQR = np.logspace(-14,-7, gridy) 
X,Y = np.meshgrid(dmu,USQR)
dmu_grid, usqr_grid = X.flatten(), Y.flatten()

In [111]:
# compute rate and lifetimes
tot_cores = mp.cpu_count()
pool = Pool(tot_cores)
############

get_event_partial = partial(hnl_tools.get_event_rate_w_mixing_and_dipole, m4 = M4, flavor_struct=[0.0,1.0,0.0],  detector = nd280)
event_rate_dipole = pool.map(get_event_partial, ((dmu, usqr) for dmu, usqr in zip(dmu_grid, usqr_grid)))

############
pool.terminate()
pool.restart()

<multiprocess.pool.Pool state=RUN pool_size=8>

In [112]:
# Save to file
table_T2K_mag_mom = np.array([dmu_grid, usqr_grid,  event_rate_dipole])
np.save("limits_computed/T2K_schwetz_mag_mom_usqr",table_T2K_mag_mom)
np.savetxt("limits_computed/T2K_schwetz_mag_mom_usqr.dat",table_T2K_mag_mom, header=" dmu(GeV^-1) USQR event_rate")


# PS191

In [113]:
# compute rate and lifetimes
tot_cores = mp.cpu_count()
pool = Pool(tot_cores)
############

get_event_partial = partial(hnl_tools.get_event_rate_w_mixing_and_dipole, m4 = M4, flavor_struct=[0.0,1.0,0.0], detector = ps191)
event_rate_dipole = pool.map(get_event_partial, ((dmu, usqr) for dmu, usqr in zip(dmu_grid, usqr_grid)))

############
pool.terminate()
pool.restart()

<multiprocess.pool.Pool state=RUN pool_size=8>

In [114]:
# Save to file
table_PS191_mag_mom = np.array([dmu_grid, usqr_grid,  event_rate_dipole])
np.save("limits_computed/PS191_schwetz_mag_mom_usqr",table_PS191_mag_mom)
np.savetxt("limits_computed/PS191_schwetz_mag_mom_usqr.dat",table_PS191_mag_mom, header=" dmu(GeV^-1) USQR event_rate")

# Dark Z' case

In [14]:
# set up grid
gridx = 100
gridy = 50

M4 = np.linspace(0.02,(m_neutral_kaon - m_mu), gridx)
USQR = np.logspace(-11,-0.5, gridy) 
X,Y = np.meshgrid(M4,USQR)
m4_grid, usqr_grid = X.flatten(), Y.flatten()

dark_coupl = {'GX': Gf*1000}

In [15]:
# compute rate and lifetime
tot_cores = mp.cpu_count()
pool = Pool(tot_cores)
############

get_event_partial = partial(hnl_tools.get_event_rate, flavor_struct=[0.0,1.0,0.0], dark_coupl=dark_coupl, detector = nd280)
event_rate_darknus = pool.map(get_event_partial, ((m4, usqr) for m4, usqr in zip(m4_grid, usqr_grid)))
get_lifetime_partial = partial(hnl_tools.get_lifetime, flavor_struct=[0.0,1.0,0.0], dark_coupl=dark_coupl)
lifetime_darknus = pool.map(get_lifetime_partial, ((m4, usqr) for m4, usqr in zip(m4_grid, usqr_grid)))

############
pool.terminate()
pool.restart()

<multiprocess.pool.Pool state=RUN pool_size=8>

In [16]:
# Save to file
table_T2K_zprime = np.array([m4_grid, usqr_grid, lifetime_darknus, event_rate_darknus])
np.save("limits_computed/T2K_zprime_Umu4",table_T2K_zprime)
np.savetxt("limits_computed/T2K_zprime_Umu4.dat",table_T2K_zprime, header=" m4(GeV) USQR tau0(s) event_rate")

In [17]:
# compute rate and lifetime
tot_cores = mp.cpu_count()
pool = Pool(tot_cores)
############

get_event_partial = partial(hnl_tools.get_event_rate, flavor_struct=[0.0,1.0,0.0], dark_coupl=dark_coupl, detector = ps191)
event_rate_darknus_ps191 = pool.map(get_event_partial, ((m4, usqr) for m4, usqr in zip(m4_grid, usqr_grid)))
get_lifetime_partial = partial(hnl_tools.get_lifetime, flavor_struct=[0.0,1.0,0.0], dark_coupl=dark_coupl)
lifetime_darknus_ps191 = pool.map(get_lifetime_partial, ((m4, usqr) for m4, usqr in zip(m4_grid, usqr_grid)))

############
pool.terminate()
pool.restart()

/Users/matheushostert/Repos/stdHNL/hnl_apps/const.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  return 1.0/G*invGeV_to_s
/Users/matheushostert/Repos/stdHNL/hnl_apps/const.py:144: RuntimeWarning: divide by zero encountered in double_scalars
  return 1.0/G*invGeV_to_cm
/Users/matheushostert/Repos/stdHNL/hnl_apps/const.py:144: RuntimeWarning: divide by zero encountered in double_scalars
  return 1.0/G*invGeV_to_cm
/Users/matheushostert/Repos/stdHNL/hnl_apps/const.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  return 1.0/G*invGeV_to_s
/Users/matheushostert/Repos/stdHNL/hnl_apps/const.py:144: RuntimeWarning: divide by zero encountered in double_scalars
  return 1.0/G*invGeV_to_cm
/Users/matheushostert/Repos/stdHNL/hnl_apps/const.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  return 1.0/G*invGeV_to_s
/Users/matheushostert/Repos/stdHNL/hnl_apps/const.py:144: RuntimeWarning: divide by zero encountered in double_scalars
  

<multiprocess.pool.Pool state=RUN pool_size=8>

In [18]:
# Save to file
table_PS191_zprime = np.array([m4_grid, usqr_grid, lifetime_darknus_ps191, event_rate_darknus_ps191])
np.save("limits_computed/PS191_zprime_Umu4",table_PS191_zprime)
np.savetxt("limits_computed/PS191_zprime_Umu4.dat",table_PS191_zprime, header=" m4(GeV) USQR tau0(s) event_rate")

# Axion-like-particle

In [37]:
# set up grid
gridx = 100
gridy = 100

M4 = np.linspace(0.02,(m_neutral_kaon - m_mu), gridx)
USQR = np.logspace(-14,-4, gridy) 
X,Y = np.meshgrid(M4,USQR)
m4_grid, usqr_grid = X.flatten(), Y.flatten()

dark_coupls = {'inv_f_alp': 1e-2, 'c_e': 1, 'c_N': 0.4, 'm_alp': 0.02}

In [38]:
# compute rate and lifetimes
tot_cores = mp.cpu_count()
pool = Pool(tot_cores)
############

get_event_partial = partial(hnl_tools.get_event_rate, flavor_struct=[0.0,1.0,0.0], dark_coupl=dark_coupls, detector = nd280)
event_rate_ALP = pool.map(get_event_partial, ((m4, usqr) for m4, usqr in zip(m4_grid, usqr_grid)))

get_lifetime_partial = partial(hnl_tools.get_lifetime, flavor_struct=[0.0,1.0,0.0], dark_coupl=dark_coupls)
lifetime_ALP = pool.map(get_lifetime_partial, ((m4, usqr) for m4, usqr in zip(m4_grid, usqr_grid)))

############
pool.terminate()
pool.restart()

<multiprocess.pool.Pool state=RUN pool_size=8>

In [39]:
# Save to file
table_T2K_alp = np.array([m4_grid, usqr_grid, lifetime_ALP, event_rate_ALP])
np.save("limits_computed/T2K_alp_Umu4", table_T2K_alp)
np.savetxt("limits_computed/T2K_alp_Umu4.dat",table_T2K_alp, header=" m4(GeV) USQR tau0(s) event_rate")

In [40]:
hnl_tools.get_event_rate([1e-2,1e-8], flavor_struct=[0.0,1.0,0.0], dark_coupl=dark_coupls, detector = ps191)

2.6633160629301595e-12

In [41]:
# compute rate and lifetime
tot_cores = mp.cpu_count()
pool = Pool(tot_cores)
############

get_event_partial = partial(hnl_tools.get_event_rate, flavor_struct=[0.0,1.0,0.0], dark_coupl=dark_coupls, detector = ps191)
event_rate_ALP_ps191 = pool.map(get_event_partial, ((m4, usqr) for m4, usqr in zip(m4_grid, usqr_grid)))

get_lifetime_partial = partial(hnl_tools.get_lifetime, flavor_struct=[0.0,1.0,0.0], dark_coupl=dark_coupls)
lifetime_ALP_ps191 = pool.map(get_lifetime_partial, ((m4, usqr) for m4, usqr in zip(m4_grid, usqr_grid)))

############
pool.terminate()
pool.restart()

<multiprocess.pool.Pool state=RUN pool_size=8>

In [42]:
# Save to file
table_PS191_ALP = np.array([m4_grid, usqr_grid, lifetime_ALP_ps191, event_rate_ALP_ps191])
np.save("limits_computed/PS191_ALP_Umu4",table_PS191_ALP)
np.savetxt("limits_computed/PS191_ALP_Umu4.dat",table_PS191_ALP, header=" m4(GeV) USQR tau0(s) event_rate")

## ALP -- mix versus 1/fa

In [43]:
# set up grid
gridx = int(100)
gridy = int(100)
M4 = 0.380
dark_coupls = {'inv_f_alp': 1e-2, 'c_e': 1, 'c_N': 0.4, 'm_alp': 0.02}

inv_fa = np.logspace(-3,-0.5, gridx)
USQR = np.logspace(-13,-5, gridy) 
X,Y = np.meshgrid(inv_fa,USQR)
inv_fa_grid, usqr_grid = X.flatten(), Y.flatten()


In [44]:
# compute rate and lifetimes
tot_cores = mp.cpu_count()
pool = Pool(tot_cores)
############

get_event_partial = partial(hnl_tools.get_event_rate_w_mixing_and_inv_fa, m4 = M4, flavor_struct=[0.0,1.0,0.0], dark_coupl=dark_coupls, detector=nd280)
event_rate_alp = pool.map(get_event_partial, ((inv_fa, usqr) for inv_fa, usqr in zip(inv_fa_grid, usqr_grid)))

############
pool.terminate()
pool.restart()

<multiprocess.pool.Pool state=RUN pool_size=8>

In [45]:
# compute rate and lifetimes
tot_cores = mp.cpu_count()
pool = Pool(tot_cores)
############

get_event_partial = partial(hnl_tools.get_event_rate_w_mixing_and_inv_fa, m4 = M4, flavor_struct=[0.0,1.0,0.0], dark_coupl=dark_coupls, detector=ps191)
event_rate_alp_ps191 = pool.map(get_event_partial, ((inv_fa, usqr) for inv_fa, usqr in zip(inv_fa_grid, usqr_grid)))

############
pool.terminate()
pool.restart()

<multiprocess.pool.Pool state=RUN pool_size=8>

In [46]:
# Save to file
table_ALP = np.array([inv_fa_grid, usqr_grid,  event_rate_alp])
np.save("limits_computed/T2K_alp_fa_usqr",table_ALP)
np.savetxt("limits_computed/T2K_alp_fa_usqr.dat",table_ALP, header=" 1/fa(GeV^-1) USQR event_rate")

In [47]:
# Save to file
table_PS191_ALP = np.array([inv_fa_grid, usqr_grid,  event_rate_alp_ps191])
np.save("limits_computed/PS191_alp_fa_usqr",table_PS191_ALP)
np.savetxt("limits_computed/PS191_alp_fa_usqr.dat",table_PS191_ALP, header=" 1/fa(GeV^-1) USQR event_rate")